# Simple Taylor method for integrating ODEs

In this notebook, we show how `TaylorSeries` can be applied in a relatively simple way to implement the Taylor method of integration for ODEs to arbitrarily high order.

This is a pedagogical example, since the Picard integration method used is not efficient in practice. Nonetheless it is a very intuitive method.

## Picard integration

We first implement Picard integration for a single-variable Taylor series:

In [ ]:
using TaylorSeries

In [ ]:
∫⬩dt(u::Taylor1) = integrate(u)

In [ ]:
function taylor_step(f, u0)

    u = copy(u0)
    unew = u0 + ∫⬩dt(f(u))

    while unew != u
        u = unew
        unew = u0 + ∫⬩dt(f(u))   # Picard iteration
    end
    
    return u
end

In [ ]:
f(x) = x

degree = 3

u0 = Taylor1([1.0], degree)

soln = taylor_step(f, u0)

In [ ]:
soln(1.0)

In [ ]:
degree = 10
u0 = Taylor1([1.0], degree)

soln = taylor_step(f, u0)

In [ ]:
soln(1.0) - exp(1.0)

In [ ]:
degree = 20 
u0 = Taylor1([1.0], degree)

soln = taylor_step(f, u0)
soln(1.0) - exp(1.0)

## More variables

Now let `u0` be a vector in $\mathbb{R}^d$ and `f` a function from $\mathbb{R}^d \to \mathbb{R}^d$. In Julia we just have to broadcast the operations using `.`:

In [ ]:
function taylor_step(f, u0)

    u = copy(u0)
    unew = u0 .+ ∫⬩dt.(f(u))

    while unew != u
        u = unew
        unew = u0 .+ ∫⬩dt.(f(u))
    end
    
    return u
end

An alternative would be to overload `∫⬩dt` to act on a vector and return a vector.

Note that this version of taylor_integration also works for a single variable, so it is in fact the only version that we need:

In [ ]:
degree = 20
u0 = Taylor1([1.0], degree)

soln = taylor_step(f, u0)
soln(1.0) - exp(1.0)

## Taylor integration

In [ ]:
TaylorSeries.displayBigO(false)

Let's run several time steps to perform a complete ODE integration:

In [ ]:
function taylor_integrate(f, x00, t0, δt, t_final, degree=10)
    
    ts = [t0]
    xs = [x00]

    # Taylor version of initial condition for each step:
    x0 = Taylor1.(x00, degree)  
    t = t0
    
    
    while t < t_final

        x = taylor_step(f, x0)
        t += δt
        
        # evaluate Taylor series at end of step to get new initial condition: 
        xend = x(δt)

        push!(ts, t)
        push!(xs, xend)
        
        # Taylor series for new initial condition:
        x0 = Taylor1.(xend, degree)
    end
    
    return ts, xs
    
end

Single variable:

In [ ]:
f(β, x) = -β * x

x0 = 1.0
t0 = 0.0
δt = 0.1
t_final = 5.0

α = 0.5
@time ts, xs = taylor_integrate(x -> f(α, x), x0, t0, δt, t_final)

In [ ]:
using Plots

In [ ]:
plot(ts, xs)
plot!(ts, t->exp(-α*t))

In [ ]:
xs[end] - exp(-α * ts[end])

### Van der Pol oscillator:

In [ ]:
van_der_Pol(μ, xx) = ( (x, y) = xx; [y, μ * (1 - x*x) * y - x] )

x0 = [1.0, 0.0]
t0 = 0.0
δt = 0.01
t_final = 20.0
degree = 10

μ = 3.0
@time ts, soln = taylor_integrate(x -> van_der_Pol(μ, x), x0, t0, δt, t_final, degree)

In [ ]:
using Plots

In [ ]:
plot(first.(soln), last.(soln), leg=false)
scatter!([first(soln[1])], [last(soln[1])])

In [ ]:
x0 = [2.0, 5.0]

@time ts, soln = taylor_integrate(x -> van_der_Pol(μ, x), x0, t0, δt, t_final, degree)

In [ ]:
plot!(first.(soln), last.(soln), leg=false)
scatter!([first(soln[1])], [last(soln[1])])

## Obtaining performance

The above is a pedagogical approach. For efficiency, we should avoid re-calculating those Taylor coefficients that have previously been calculated in each step of the Picard integration. One solution for this is the `@taylorize` macro in `TaylorIntegration.jl`.